In [1]:
import os
import pandas as pd
import numpy as np
import re
from catboost import CatBoostClassifier

In [2]:
from splitting import combine_files_into_dfs

In [3]:
FOLDER = '/home/alexey.osipov/playground/eco_geo/prototype/'

In [4]:
X_train, y_train = combine_files_into_dfs('train.txt', FOLDER)

In [5]:
X_train.drop(['Line'], inplace = True, axis = 1)
y_train = y_train['Dep_labels']

In [6]:
model_1 = CatBoostClassifier(iterations=10)

In [7]:
model_1.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.0272480	total: 484ms	remaining: 4.36s
1:	learn: 0.0130553	total: 734ms	remaining: 2.94s
2:	learn: 0.0097539	total: 1.05s	remaining: 2.45s
3:	learn: 0.0075127	total: 1.45s	remaining: 2.17s
4:	learn: 0.0062065	total: 1.76s	remaining: 1.76s
5:	learn: 0.0051866	total: 2.07s	remaining: 1.38s
6:	learn: 0.0042602	total: 2.4s	remaining: 1.03s
7:	learn: 0.0036915	total: 2.7s	remaining: 676ms
8:	learn: 0.0032663	total: 2.99s	remaining: 333ms
9:	learn: 0.0029297	total: 3.3s	remaining: 0us


In [8]:
def blend_file(filename = 'pred_data_8_56.csv',
               foldername = '/home/alexey.osipov/playground/eco_geo/prototype/data/',
               targetname = 'Dep_labels',
               keycolumns = ['X', 'Y', 'f4_AuAs', 'f7_AuAgAs_WCuMo', 'Gr_4_5'],
               model1 = model_1):
    raw_filename = re.sub('pred_', '', filename)
    tab_features = pd.read_csv(foldername + 'features/' + raw_filename)
    tab_features = tab_features.iloc[:,2:]
    tab_features_filtered = tab_features[keycolumns]
    tab_masks = pd.read_csv(foldername + 'masks/' + re.sub('data', 'masks', raw_filename))
    tab_results_deeplab_1 = pd.read_csv(foldername + 'results/' + filename)
    tab_results_deeplab_2 = pd.read_csv(foldername + 'results_external/' + filename)
    tab_masks_deeplab = pd.read_csv(foldername + 'results/' + re.sub('pred_', 'target_', filename))
    assert np.all(tab_masks[targetname].values == tab_masks_deeplab['target'].values)
    res1 = model1.predict(tab_features)
    tab_results_deeplab_1['model1'] = res1
    tab_results_deeplab_1['model2'] = tab_results_deeplab_2['prediction'].values
    tab_results_deeplab_1['model'] = round((tab_results_deeplab_1['prediction'] +\
        tab_results_deeplab_1['model1'] + tab_results_deeplab_1['model2'])/3)
    tab_results_deeplab_1['prediction'] = tab_masks_deeplab['target']
    numerator = len(tab_results_deeplab_1[(tab_results_deeplab_1['model'] == 1) & (tab_results_deeplab_1['prediction'] == 1)])
    denominator = len(tab_results_deeplab_1[(tab_results_deeplab_1['model'] == 1) | (tab_results_deeplab_1['prediction'] == 1)])
    return numerator, denominator

In [9]:
file_list = os.listdir('/home/alexey.osipov/playground/eco_geo/prototype/data/results')
file_list = [f for f in file_list if 'pred_' in f]

In [10]:
numerators = []
denominators = []
for f in file_list:
    print(f)
    numerator, denominator = blend_file(f)
    numerators.append(numerator)
    denominators.append(denominator)

pred_data_39_25.csv
pred_data_55_7.csv
pred_data_48_23.csv
pred_data_69_8.csv
pred_data_15_24.csv
pred_data_16_24.csv
pred_data_19_46.csv
pred_data_3_54.csv
pred_data_21_28.csv
pred_data_46_16.csv
pred_data_22_47.csv
pred_data_21_39.csv
pred_data_18_47.csv
pred_data_59_4.csv
pred_data_57_7.csv
pred_data_30_21.csv
pred_data_22_48.csv
pred_data_20_45.csv
pred_data_71_15.csv
pred_data_52_15.csv
pred_data_28_35.csv
pred_data_50_21.csv
pred_data_42_4.csv
pred_data_8_45.csv
pred_data_7_46.csv
pred_data_9_47.csv
pred_data_54_7.csv
pred_data_33_1.csv
pred_data_27_36.csv
pred_data_49_22.csv
pred_data_22_18.csv
pred_data_58_7.csv
pred_data_27_20.csv
pred_data_33_29.csv
pred_data_8_39.csv
pred_data_39_26.csv
pred_data_27_19.csv
pred_data_56_8.csv
pred_data_61_8.csv
pred_data_22_31.csv
pred_data_47_16.csv
pred_data_63_25.csv
pred_data_40_26.csv
pred_data_6_33.csv
pred_data_29_35.csv
pred_data_32_12.csv
pred_data_33_31.csv
pred_data_15_27.csv
pred_data_19_29.csv
pred_data_43_6.csv
pred_data_30_14.c

pred_data_46_15.csv
pred_data_30_39.csv
pred_data_31_6.csv
pred_data_29_8.csv
pred_data_21_48.csv
pred_data_10_36.csv
pred_data_23_16.csv
pred_data_19_30.csv
pred_data_59_23.csv
pred_data_48_21.csv
pred_data_20_47.csv
pred_data_28_8.csv
pred_data_41_3.csv
pred_data_52_17.csv
pred_data_20_27.csv
pred_data_70_6.csv
pred_data_14_25.csv
pred_data_33_19.csv


In [11]:
sum(numerators)/sum(denominators)

0.24359093012494215